In [225]:
import pandas as pd 
import numpy as np
import shutil
import glob
import re

from os.path import exists, basename
from datetime import datetime


In [226]:
#finding Jira Pull from downloads
base_dir = "./Jira"
today = str(datetime.today().strftime("%Y-%m-%d"))
# today = 'TEST'
# today = '2021-12-17'
tracker = f"NCI Tracker {today}T"

#reporting file
reporting_file = base_dir + "/reporting.csv"
# reporting_file = base_dir + "/reporting_test.csv"

#curation steps
curation_step = "./curationSteps.csv"

#moving to workspace
try:
    tracker_path = glob.glob(f"/Users/liualg/Downloads/{tracker}*")[0]
    shutil.move(tracker_path, f"{base_dir}/pull/")
except:
    print("NCI Tracker: does not exist")

NCI Tracker: does not exist


In [227]:
def condense_labels(df):
    dfTracker = df
    pmids = []
    for count in range(len(dfTracker)):
        print(count)
        if re.match(".*PMID_.*", dfTracker['Labels'][count]):
            pmids.append(dfTracker['Labels'][count])

        elif re.match(".*PMID_.*", dfTracker['Labels.1'][count]):
            pmids.append(dfTracker['Labels.1'][count])

        elif re.match(".*PMID_.*", dfTracker['Labels.2'][count]):
            pmids.append(dfTracker['Labels.2'][count])
        else:
            print(f'{count}: no PMID')

    return(pmids)

In [228]:
def condense_labels(df):
    dfTracker = df
    pmids = []
    for count in range(len(dfTracker)):
#         print(count)
        if re.match(".+_\d+", dfTracker['Labels'][count]):
            pmids.append(dfTracker['Labels'][count])

        elif re.match(".+_\d+", dfTracker['Labels.1'][count]):
            pmids.append(dfTracker['Labels.1'][count])

        elif re.match(".+_\d+", dfTracker['Labels.2'][count]):
            pmids.append(dfTracker['Labels.2'][count])
        else:
            print(f'{count}: no PMID')

    return(pmids)

In [229]:
def flatten(A):
    rt = []
    for i in A:
        if isinstance(i,list): rt.extend(flatten(i))
        else: rt.append(i)
    return rt

In [230]:
if exists(f"{base_dir}/pull/{basename(tracker_path)}"):
    dfTracker = pd.read_csv(f"{base_dir}/pull/{basename(tracker_path)}") #new data pull
    pmids = condense_labels(dfTracker) # PMIDS for datapull
    
    if exists(reporting_file):
        print('file exists')
        report = pd.read_csv(reporting_file, index_col=0)
        print(report)
        check = 0
    else:
        reporting_dict = {
            'Percent done':['']*len(pmids),
            'PMID':pmids,
            today: dfTracker['Summary']
        }
        report = pd.DataFrame(reporting_dict)
        report.to_csv(reporting_file)
        check = 1
else:
    print('Tracker not in /Pull')

file exists
    Percent done                   PMID  \
0            NaN  Liu_Preprint_34100011   
1            NaN          PMID_34314495   
2            NaN          PMID_34605900   
3            NaN          PMID_34605900   
4            NaN          PMID_34605900   
5            NaN          PMID_33993265   
6            NaN          PMID_33993265   
7            NaN          PMID_33993265   
8            NaN          PMID_34652783   
9            NaN          PMID_34652783   
10           NaN          PMID_33622794   
11           NaN          PMID_34739850   
12           NaN          PMID_34739850   
13           NaN          PMID_34835131   
14           NaN          PMID_34835131   
15           NaN          PMID_33142304   
16           NaN          PMID_33169014   
17           NaN          PMID_33571162   
18           NaN          PMID_33795870   
19           NaN          PMID_33830946   
20           NaN          PMID_33830946   
21           NaN          PMID_33846272   

In [220]:
if check == 1:
    new_pull = pd.DataFrame({"PMID":pmids, today: dfTracker['Summary']})
    new_column = ['']*len(report['PMID'])
    new_rows = []
    used_PMID = []

    #matching PMIDs to current reporting document 
    for specific_PMID in new_pull['PMID']:
        if len(report.loc[report['PMID'] == specific_PMID]) > 0:

            #If the PMIDs exist, add to new_column list (using index)
            index_PMID = report.loc[report['PMID'] == specific_PMID].index[0]
            if specific_PMID in used_PMID:
                new_column[index_PMID] += ', ' + new_pull[today][index_PMID]
            else:
                new_column[index_PMID] = new_pull[today][index_PMID]

            used_PMID.append(specific_PMID)
        else:
            #Save the new PMIDs so we can add them as rows after 
            new_rows.append(specific_PMID)   

    report[today] = new_column
    display(report)

In [221]:
if check == 1:
    #Adding new PMIDs as rows
    for new_PMID in new_rows:
        used_PMID = []
        add_PMID = []
        #finding the index 
        index_PMID = new_pull.loc[new_pull['PMID'] == new_PMID].index[0]

        #Seeing if the row exists
        if new_PMID in used_PMID:
            add_PMID += ', ' + new_pull[today][index_PMID]
        else:
            add_PMID = new_pull[today][index_PMID]

        used_PMID.append(new_PMID)

        #Creating the row and adding it to the df
        report.loc[len(report.index)] = flatten([['Na'], new_PMID,
                                                 ['Na']*(len(report.columns)-3),
                                                 new_pull[today][index_PMID]]) 

    display(report)


#### Adding % done ####

In [222]:
step_list = pd.read_csv(curation_step)


In [223]:
percent_done = []
for current_step in report[today]:
    if len(current_step) > 1:
        current_step = current_step.split(',')
        
#     If there is only 1 current step
        if len(current_step) == 1:
            step = step_list.loc[step_list['Curation Steps'] == current_step[0]].index[0] + 1
            percent_done.append('%' + str(int(step/len(step_list)*100)))

#     Finding the index and taking the average
        elif len(current_step) > 1:
            temp = []
            for steps in current_step:
                temp.append(step_list.loc[step_list['Curation Steps'] == current_step[0]].index[0] + 1)
            percent_done.append('%' + str(int(np.mean(temp)/len(step_list)*100)))

    else:
        percent_done.append('Not In Progress')
            
report['Percent done'] = percent_done

In [224]:
report.head()

,Percent done,PMID,2022-01-13
0,%90,Liu_Preprint_34100011,ImmPort Informed
1,%5,PMID_34314495,Protocol Template
2,%25,PMID_34605900,Institutional Certification Request Sent
3,%10,PMID_34605900,POC Requested
4,%5,PMID_34605900,Protocol Template


In [153]:
# report.to_csv(reporting_file, index= False)
report.to_csv('/Users/liualg/Documents/GitHub/SeroNet/auto_add_masterlist/Jira/reporting_file.csv', index= False)